In [2]:
import pandas as pd
import plotly.express as px
import numpy as np

In [3]:
# --- Load and clean ---
gni_df = pd.read_csv("labelled_gni_data.csv")
pop_df = pd.read_csv("population_data.csv")

# Pick the most recent available GDP per capita value
gni_df["gni"] = gni_df["2024"].fillna(gni_df["2023"]).fillna(gni_df["2022"])

# Pick the most recent available population value
pop_df["population"] = pop_df["2024"].fillna(pop_df["2023"]).fillna(pop_df["2022"])

#calculate gni per capita
df = pd.merge(gni_df[["Country Name", "Country Code", "gni"]], pop_df[["Country Code", "population"]], on="Country Code")
df["gni_per_capita"] = df["gni"] / df["population"]

# convert to monthly as the LLM cost is monthly
df["gni_per_capita"] = df["gni_per_capita"] / 12

# Keep only valid ISO3 country codes (drop aggregates)
df = df[df["Country Code"].str.len() == 3]

# Drop countries with no GDP data
df = df.dropna(subset=["gni_per_capita"])

# --- Compute LLM cost burden ---
df["Gemini_AI_Ultra_cost_burden_percent"] = (250 / df["gni_per_capita"]) * 100

# Log transform (add small constant to avoid log(0))
df["Gemini_AI_Ultra_cost_burden_log"] = np.log10(df["Gemini_AI_Ultra_cost_burden_percent"] + 1e-6)

In [10]:
codes = ["USA", "BGD", "COD"]   # or "COG" depending on which Congo you want

result = df[df["Country Code"].isin(codes)][["Country Code", "gni_per_capita"]]

# multiply by 12 to get annual amount
result["gni_per_capita_annual"] = result["gni_per_capita"] * 12

print(result)


    Country Code  gni_per_capita  gni_per_capita_annual
20           BGD      838.517042           10062.204500
43           COD      137.738053            1652.856638
251          USA     7165.087533           85981.050398


In [27]:
# --- Plot ---
fig = px.choropleth(
    df,
    locations="Country Code",
    color="Gemini_AI_Ultra_cost_burden_log",
    hover_name="Country Name",
    hover_data={"Gemini_AI_Ultra_cost_burden_percent": ":.2f"},
    color_continuous_scale="Viridis",   # good for log data
    title="Log-scaled Gemini AI Ultra Cost Monthly Burden (% of GNI per Capita Monthly for $250)",
    width=1800,
    height=1080,
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True, projection_type="equirectangular")
)

fig.show()